In [3]:
from gensim import corpora,models,similarities
import jieba
from collections import defaultdict
import urllib.request
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')  #后两行主要解决gensim warning

# import 中的一些问题

## [关于defaultdict](http://www.zlovezl.cn/articles/collections-in-python/)

## [关于gensim warning](https://stackoverflow.com/questions/41658568/chunkize-warning-while-installing-gensim)

# 对要计算的多篇文档进行分词

In [4]:
with open(r'gcd.txt',encoding = 'utf-8') as f:      #鬼吹灯
    d1 = f.read()
with open(r'dmbj.txt',encoding = 'utf-8') as f:     #盗墓笔记
    d2 = f.read()

In [5]:
data1 = jieba.cut(d1)
data2 = jieba.cut(d2)

In [6]:
df1 = ""
for item in data1:
    df1 += item + " "
df2 = ""
for item in data2:
    df2 += item + " "

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Kismet\AppData\Local\Temp\jieba.cache
Loading model cost 0.960 seconds.
Prefix dict has been built succesfully.


In [7]:
docs = [df1,df2]
texts = [[word for word in doc.split()] for doc in docs]

## 过滤掉低频词汇的写法

In [ ]:
'''
texts=[[word for word in text if frequency[token]>3]
 for text in texts]
'''

# 计算出词语的频率

In [9]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token]+= 1

# 通过语料库建立词典

In [12]:
dictionary = corpora.Dictionary(texts)
dictionary.save("dict_0817.txt")

# 加载要对比的文档

In [15]:
with open(r'ljm.txt',encoding = 'utf-8') as f:     #老九门
    d3 = f.read()

In [21]:
data3 = jieba.cut(d3)
df3 = ""
for item in data3:
    df3 += item + " "

# 将要对比的文档通过doc2bow转化为稀疏向量，对稀疏向量进一步处理，得到新的语料库

In [23]:
new_doc = df3
new_vec = dictionary.doc2bow(new_doc.split())
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize("dict_0817,txt",corpus)

# 将新语料库通过tf-idf model进行处理，得到tf-idf,并通过token2id得到特征数

In [24]:
tf_idf = models.TfidfModel(corpus)
featureNum = len(dictionary.token2id.keys())

# 稀疏矩阵相似度，从而建立索引

In [ ]:
index = similarities.SparseMatrixSimilarity(tf_idf[corpus],num_features = featureNum)
simi = index[tf_idf[new_vec]]

# 得到最终相似度结果

In [25]:
simi

array([ 0.02089206,  0.02730641], dtype=float32)